In [ ]:
import llvmlite.binding as llvm
import os
import subprocess
import sys
import os
import tempfile
import shutil

In [59]:
llvm_dis_path='/usr/lib/llvm-15/bin/llvm-dis'
opt_path='/usr/lib/llvm-15/bin/opt'
clang_path='/usr/lib/llvm-15/bin/clang'
c_file_path = r'/home/xucong24/Compiler/datasets/qsort.c'
opt_flags = ["-adce", "-instcombine", "-simplifycfg", "-mem2reg"]

In [60]:
temp_dir = '/home/xucong24/Compiler/tmp'
bc_file = os.path.join(temp_dir, "output.bc")
opt_bc_file = os.path.join(temp_dir, "optimized.bc")
opt_ll_file = os.path.join(temp_dir, "optimized.ll")

In [61]:
subprocess.run([
    clang_path, 
    "-S", 
    "-emit-llvm", 
    "-o", bc_file, 
    c_file_path, 
    "-O1", 
    "-Xclang", "-disable-llvm-passes", 
    "-Xclang", "-disable-llvm-optzns",
    "-isystem", "/usr/include/c++/11",
    "-isystem", "/usr/include/x86_64-linux-gnu/c++/11",
    "-isystem", "/usr/include/c++/11/backward",
    "-isystem", "/usr/lib/gcc/x86_64-linux-gnu/11/include",
    "-isystem", "/usr/local/include",
    "-isystem", "/usr/include/x86_64-linux-gnu",
    "-isystem", "/usr/include"
], check=True)

CompletedProcess(args=['/usr/lib/llvm-15/bin/clang', '-S', '-emit-llvm', '-o', '/home/xucong24/Compiler/tmp/output.bc', '/home/xucong24/Compiler/datasets/qsort.c', '-O1', '-Xclang', '-disable-llvm-passes', '-Xclang', '-disable-llvm-optzns', '-isystem', '/usr/include/c++/11', '-isystem', '/usr/include/x86_64-linux-gnu/c++/11', '-isystem', '/usr/include/c++/11/backward', '-isystem', '/usr/lib/gcc/x86_64-linux-gnu/11/include', '-isystem', '/usr/local/include', '-isystem', '/usr/include/x86_64-linux-gnu', '-isystem', '/usr/include'], returncode=0)

In [ ]:
/usr/lib/llvm-15/bin/clang -S -emit-llvm -opaque-pointers /home/xucong24/Compiler/datasets/qsort.c -o /home/xucong24/Compiler/tmp/output.ll

In [62]:
subprocess.run([opt_path, *opt_flags, bc_file, "-o", opt_bc_file], check=True)
subprocess.run([llvm_dis_path, opt_bc_file, "-o", opt_ll_file], check=True)


CompletedProcess(args=['/usr/lib/llvm-15/bin/llvm-dis', '/home/xucong24/Compiler/tmp/optimized.bc', '-o', '/home/xucong24/Compiler/tmp/optimized.ll'], returncode=0)

In [69]:
with open('/home/xucong24/Compiler/tmp/output.ll', "r") as f:
    llvm_ir = f.read()

In [70]:
llvm.initialize()
llvm.initialize_native_target()
llvm.initialize_native_asmprinter()
# llvm.initialize_all_targets()
# llvm.initialize_all_asmprinters()

# 解析字符串中的LLVM IR
module = llvm.parse_assembly(llvm_ir)

# 计算指令数量
instruction_count = 0
for function in module.functions:
    for block in function.blocks:
        for instruction in block.instructions:  # 使用 .instructions 获取块中的指令
            instruction_count += 1
            
instruction

<string>:12:34: warning: ptr type is only supported in -opaque-pointers mode
define dso_local void @quickSort(ptr noundef %0, i32 noundef %1, i32 noundef %2) #0 {
                                 ^


RuntimeError: LLVM IR parsing error
<string>:12:34: error: expected type
define dso_local void @quickSort(ptr noundef %0, i32 noundef %1, i32 noundef %2) #0 {
                                 ^


In [ ]:
opt_flags = ["-adce", "-instcombine", "-simplifycfg", "-mem2reg"]

llvm_ir = compile_c_to_llvm_ir(c_file_path, llvm_path='/usr/lib/llvm-15/bin')
print(llvm_ir)
print(f"优化前IR指令数量: {count_llvm_ir_instructions(llvm_ir)}")
llvm_ir = compile_c_to_llvm_ir(c_file_path, opt_flags=opt_flags, llvm_path='/usr/lib/llvm-15/bin')
print(f"优化后IR指令数量: {count_llvm_ir_instructions(llvm_ir)}")